# Introduction to Quantum Machine Learning

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from qibo.models import Circuit
from qibo import gates, hamiltonians, set_backend
from qibo.optimizers import optimize

set_backend("numpy")

## 1. A look back to quantum circuits

In particular we will focus on parametric circuits, because we want to use them as machine learning models.

A rotational gate can be added to a `qibo` circuit using the command: `circuit.add(qibo.gates.RX(q=q, theta=theta))`,
in which you set the ID of the qubit `q` and the angle `theta`.

### Building a simple 1-qubit circuit

We start building a simple 1-qubit circuit:

In [ ]:
# define a circuit
c = Circuit(1)

# we can add parametric gates
c.add(gates.RY(q=0, theta=0))
c.add(gates.RZ(q=0, theta=0))

# and a measurement gate
c.add(gates.M(0))

whose information can be printed as follows:

In [ ]:
# circuit drawing
print(c.draw())

# and circuit information
print(f"\n{c.summary()}")

We can also execute the circuit, obtaining a state, in which we collect some informations, like amplitudes and frequencies.

In [ ]:
# circuit execution
final_state = c.execute(nshots=1000)
print(f"\nfinal state: {final_state}")

# print probabilities
print(f"\nprobabilities: {final_state.probabilities(qubits=[0])}")

# print frequencies
print(f"\nprobabilities: {final_state.frequencies()}")

### A function for printing frequencies

In [ ]:
def visualize_states(counter, ticks_rotation=0):
    """States visualization."""
 
    fig, ax = plt.subplots(figsize=(10,5))

    ax.set_title('State visualization')
    ax.set_xlabel('States')
    ax.set_ylabel('#')

    for state in counter:
        ax.scatter(state, counter[state], color='purple', alpha=0.5, s=150)
        ax.vlines(state, 0, counter[state] - 12, color='black', ls='-', lw=1.2)
        
    plt.xticks(rotation=ticks_rotation)
    
    plt.grid(True)

In [ ]:
visualize_states(final_state.frequencies())

### Modifying circuits parameters

We can modify the circuit's parameters and have access to this information using the following commands:

- `circuit.get_parameters()` to get the parameters;
- `circuit.set_parameters(new_params)` to set `new_params` as circuit parameters.

In [ ]:
# set new angles in the rotations
nparams = len(c.get_parameters())
print(f"\nnparams: {nparams}")
print(f"params: {c.get_parameters()}")

In [ ]:
# set a new parameter
c.set_parameters(np.random.randn(nparams))
print(f"new params: {c.get_parameters()}")

# circuit execution with new params
final_state = c.execute(nshots=1000)
print(f"\nfinal state: {final_state}")

In [ ]:
# print probabilities
print(f"\nprobabilities: {final_state.probabilities(qubits=[0])}")

# print frequencies
print(f"\nprobabilities: {final_state.frequencies()}")

In [ ]:
visualize_states(final_state.frequencies())

### Defining an Hamiltonian

We are going to define target Hamiltonians in order to use their expected value over some final state as QML predictor.
An Hamiltonian can be defined as follows with `Qibo`:

In [ ]:
# set hamiltonian
h = hamiltonians.Z(nqubits=1) 
print(f"Hamiltonian:\n{h.matrix}")

# expectation
print(f"\nExpectation: {h.expectation(c.execute().state())}")

## 2. Play with parametric gates

In [ ]:
x_angles = np.linspace(-2*np.pi, 2*np.pi, 100)
y_angles = np.linspace(0, np.pi, 100)

expectations = []

for x, y in zip(x_angles, y_angles):
    c.set_parameters([x, y])
    expectations.append(h.expectation(c.execute().state()))

In [ ]:
def plot_expectations(energies):
    """Plot energy in function of the epochs"""
    plt.figure(figsize=(8,5))
    plt.title("Energy over training")
    plt.plot(energies, color="purple", alpha=0.7, lw=2, label="Energy values")
    plt.xlabel(r"$\theta$")
    plt.ylabel("E")
    plt.grid(True)
    plt.legend()
    plt.show()

In [ ]:
plot_expectations(expectations)

## 3. Injecting information into a circuit

We can define a big class of parametric circuits, which can be called Variational 
Quantum Circuits (VQC), in whose parametric gates we can inject **both** data and 
variational parameters!

<img src="figures/vqc.png" width="600" height="600" align="center"/>

One way to embed data in QML is to define some uploading layer into the circuit and then to 
repeat the uploading of the data many times [1]. 

In [ ]:
# define a circuit composed of nlayers

def build_vqc(nqubits, nlayers):
    """Build VQC composed of nqubits and nlayers"""
    
    # init circuit
    vqc = Circuit(nqubits)
    
    # loop over layers
    for l in range(nlayers):
        # loop over qubits
        for q in range(nqubits):
            vqc.add(gates.RY(q=q, theta=0))
            vqc.add(gates.RY(q=q, theta=0))
            vqc.add(gates.RZ(q=q, theta=0))
            vqc.add(gates.RZ(q=q, theta=0))
        # we add an entangling channel at the end of each layer
        for q in range(0, nqubits-1):
            vqc.add(gates.CNOT(q0=q, q1=q+1))
        vqc.add(gates.CNOT(q0=nqubits-1, q1=0))
    # we add one measurement gate for each qubit
    vqc.add(gates.M(*range(nqubits)))
    
    return vqc

In the previous VQC definition, some CNOT gates are appended to the circuit. These gates are super important to create **entanglement** in the system, as you already know from the previous Quantum Computing tutorial!

In [ ]:
nqubits = 4
nlayers = 3

vqc = build_vqc(nqubits=nqubits, nlayers=nlayers)
print(vqc.draw())

### Combine data and parameters while filling the circuit

In [ ]:
# define a way for uploading both data and parameters

def inject_data(circuit, nlayers, parameters, x):
    """Inject data and params into the circuit."""
    
    # empty list of parameters
    params = []
    # we keep track of the index
    index = 0

    # loop over layers
    for l in range(nlayers):
        # loop over qubits
        for q in range(circuit.nqubits):
            # we fill the first RY with param * x
            params.append(parameters[index] * x)
            # bias
            params.append(parameters[index + 1])
            # we fill the first RZ with param * x
            params.append(parameters[index + 2] * x)
            # bias
            params.append(parameters[index + 3])

            # update index counter to prepare the next 4 parameters
            index += 4
    
    # set the new parameters affected by x into the circuit and return it
    circuit.set_parameters(params)
    return circuit

In [ ]:
# set random parameters inside the circuit
np.random.seed(42)
old_params = np.random.randn(len(vqc.get_parameters()))
print(old_params)

In [ ]:
# inject data
x = 2

vqc = inject_data(circuit=vqc, nlayers=nlayers, parameters=old_params, x=x)

# get new params after the injection
new_params = vqc.get_parameters()

# sanity check
print("Check the even params are doubled:\n")
for p in range(8):
    print(f"Old value: {old_params[p]:.4}\t New value: {new_params[p][0]:.4}")

### Check the final state 

In [ ]:
# final state 
fstate = vqc.execute(nshots=1000)

# frequencies
visualize_states(fstate.frequencies(), ticks_rotation=60)

## 4. A snapshot of quantum machine learning

<img src="figures/qml.png" width="1000" height="1000" align="center"/>


## 5. Exercise: tune circuit parameters to get a target value

Define:
1. a 1-qubit circuit with two parametrized gates: an RY followed by an RZ, with a measurement gate in the end;
2. an 1-qubit hamiltonian to be used as target observable: in particular I suggest you to use a pauli Z;
3. a target variable `target=0.5`;
4. initialize the two parameters of the circuit to some value (this choice should be done in a reasonable way in principle, but in case of large circuits the parameters can also be set randomly);
5. use `qibo.optimizers.optimize` module with `method="cma"` to find the optimized params and passing as loss function the one suggested some cells below (the optimize method will return also the best set of parameters);
6. compute the expected value of the hamiltonian on the state we obtain by executing the circuit filled with the best parameters returned by the CMA optimizer;

In [ ]:
# set model

# set hamiltonian

# target value

In [ ]:
def loss(parameters, hamiltonian, model, target):
    """Mean Squared Error with y_target given model and hamiltonian."""
    model.set_parameters(parameters)
    expectation = hamiltonian.expectation(model.execute().state())
    return (expectation-target)**2

## Question: can you tackle any target value using this setup?